# Introduction
This notebook shows how to create a simple but very silly strategy from scratch, the ** Werewolf Strategy **.

Our strategy adheres to the following 3 rules: 

1. If it is a super moon => we buy stocks
2. If it is a micro moon => we sell our stocks so we have no more exposure
3. Otherwise => we do nothing

Btw, a supermoon occurs when the moon’s orbit is closest to Earth at the same time it is full. So visually it is a very large moon. 

The main purpose of this example is obviously not to create a useful strategy that will make you a huge profit. But rather to demonstrate how easy it is to develop your own strategy.
As always, we start with loading roboquant using the cell magic *%use*


In [ ]:
%use roboquant(3.0.0-SNAPSHOT)
Welcome()

## Load external libraries
We can load any library from external repositories and import the required packages afterwards. If required we can even add repositories to the notebook. In the background the notebook kernel will download the required package and all its declared dependencies. 

In this case we download the *common-suncalc* package for calculating the moon phases. And because it is available in one of the standard Maven repositories, we don't need to add that first.  

In [ ]:
@file:DependsOn("org.shredzone.commons:commons-suncalc:3.4")
import org.shredzone.commons.suncalc.*

## Coding our own Strategy

We're now ready to develop the custom strategy.

In [ ]:
class WerewolfStrategy(val asset: Asset) : Strategy {

    override fun createSignals(event: Event): List<Signal> {
        event.getPrice(asset) ?: return emptyList()
        val phase = MoonPhase.compute().on(event.time).execute()
        return when {
            phase.isSuperMoon -> listOf(Signal.buy(asset))
            phase.isMicroMoon -> listOf(Signal.sell(asset))
            else -> emptyList()
        }
    }

}

## Run
We are now ready to back-test our strategy. And we decided to test it against Apple stock. So what we need to do is:

- Use a data feed that contains historic prices for the Apple stock
- Create an instance of our strategy and tell it to focus on the Apple stock
- Create a run

In [ ]:
val feed = AvroFeed.sp25()
val apple = Stock("AAPL")
val strategy = WerewolfStrategy(apple)
run(feed, strategy)